# Convolution Neural Networks with GLOVE o WORD2VEC

In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../") 

from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer
)
from sklearn.metrics import f1_score
from ast import literal_eval

from personal_library.sce_keras.callbacks import (
    F1PrecisionRecall,
    LearningRateDecay
)
from personal_library.sce_keras.loss_functions import f1_loss
from personal_library.sce_keras.metrics_functions import f1

/Users/sebastiancorrea/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/sebastiancorrea/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [45]:
x = np.load('../data/dataset/X.npy').reshape(x.shape[0], x.shape[1], x.shape[2], 1)
y = np.load('../data/dataset/y.npy')

shape = x[0].shape
x_train, x_val, y_train, y_val = train_test_split(x, y, random_state=1992, test_size=0.2)

In [ ]:
from personal_library.sce_keras.layers.custom_layers import CBGN
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential 
from keras.optimizers import Adam
from keras.layers import (
    Flatten,
    Activation,
    Dense
)


num_classes = 1
batch_size = 32
epochs = 20
learnRate = 0.001

metric = F1PrecisionRecall()
lrate_decay = LearningRateDecay(epochs, learnRate).step_decay

checkpoint_path = "../model_wehigts/conv_wehigts.hdf5"
checkpointer = ModelCheckpoint(filepath=checkpoint_path,
                               monitor='val_loss', verbose=1,
                               save_best_only=True, mode='min')

model = Sequential()
model=CBGN(model,16, shape)
model=CBGN(model,32)
model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid'))

adam = Adam(lr=learnRate, beta_1=0.9, beta_2=0.999,
            epsilon=None, decay=1e-6, amsgrad=False)

# 'binary_crossentropy'
model.compile(loss=f1_loss, 
              optimizer='adam', 
              metrics=['accuracy', f1]) 

history = model.fit(x_train, y_train, 
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[checkpointer, lrate_decay])

Train on 25569 samples, validate on 6393 samples
Epoch 1/20
19008/25569 [=====================>........] - ETA: 3:00 - loss: 0.8749 - acc: 0.0719 - f1: 0.1251

In [29]:
np.unique(y_val)

array([0.0, 1.0], dtype=object)

In [33]:
y_train

array([0.0, 0.0, 0.0, ..., 1.0, 0.0, 0.0], dtype=object)